In [30]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras import models, layers, activations, initializers, losses, optimizers, metrics
from keras.models import Sequential, Model
from keras.layers import Input, LeakyReLU, BatchNormalization, LSTM
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.layers.core import Dense, Activation, Dropout

In [31]:
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    length = len(sequence)
    col_length = len(sequence[0])
    for i in range(length):
        end_ix = i + n_steps
        if end_ix > length-1:
            break
        seq_x, seq_y = sequence[i:end_ix, :], sequence[end_ix, col_length-1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [32]:
data = pd.read_csv('data/samsung.csv', encoding='euc-kr')
data = data.sort_values(['일자'], ascending=[True])

data = data.iloc[:,1:].copy()

data = pd.concat([data.iloc[:,:3], data.iloc[:,4], data.iloc[:,3]], axis=1)

for i in data.columns:
    data[i] = data[i].apply(lambda x: x.replace(",","",-1))

for i in data.columns:
    data[i] = data[i].astype(int)

data = np.array(data)

In [33]:
n_steps = 5
x_data, y_data = split_sequence(data, n_steps)

In [34]:
x_data = x_data.reshape((x_data.shape[0], -1))
scaler = StandardScaler()
scaler.fit(x_data)
x_data = scaler.transform(x_data)
x_data = x_data.reshape((x_data.shape[0], -1, 1))
x_data.shape

(421, 25, 1)

In [35]:
final_predict_data = x_data[-1:,:].copy()

In [36]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_data, y_data, train_size=0.9, random_state=66, shuffle=True
)

In [37]:
inp = Input(shape=(x_train.shape[1],x_train.shape[2]))
x1 = LSTM(128, activation='relu')(inp)
x1 = Dense(256)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.4)(x1)
x1 = Dense(128)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
x1 = Dropout(0.25)(x1)
x1 = Dense(64)(x1)
x1 = LeakyReLU(alpha=0.05)(x1)
out = Dense(1)(x1)

model = Model(inputs=inp, outputs=out)
# 훈련
model.compile(loss='mse', optimizer='adam', metrics=['acc'])
model.fit(x_train, y_train, epochs=100, batch_size=1)

Epoch 1/100
378/378 [==============================] - 5s 12ms/step - loss: 11860348844.8677 - acc: 0.0000e+00
Epoch 2/100
378/378 [==============================] - 4s 10ms/step - loss: 2189718598.7725 - acc: 0.0000e+00
Epoch 3/100
378/378 [==============================] - 4s 10ms/step - loss: 16151063008.1270 - acc: 0.0000e+00
Epoch 4/100
378/378 [==============================] - 4s 10ms/step - loss: 2088477888.0000 - acc: 0.0000e+00
Epoch 5/100
378/378 [==============================] - 4s 10ms/step - loss: 2043442148.2328 - acc: 0.0000e+00
Epoch 6/100
378/378 [==============================] - 4s 10ms/step - loss: 1949149294.5608 - acc: 0.0000e+00
Epoch 7/100
378/378 [==============================] - 4s 10ms/step - loss: 9181949218.2857 - acc: 0.0000e+00
Epoch 8/100
378/378 [==============================] - 4s 10ms/step - loss: 1701065331.8095 - acc: 0.0000e+00
Epoch 9/100
378/378 [==============================] - 4s 10ms/step - loss: 1403455932658981.2500 - acc: 0.0000e+00
Ep

378/378 [==============================] - 5s 13ms/step - loss: 42132685.0925 - acc: 0.0000e+00
Epoch 76/100
378/378 [==============================] - 5s 13ms/step - loss: 48213487.7432 - acc: 0.0000e+00
Epoch 77/100
378/378 [==============================] - 5s 13ms/step - loss: 36644512.6268 - acc: 0.0000e+00
Epoch 78/100
378/378 [==============================] - 5s 14ms/step - loss: 41827663.5782 - acc: 0.0000e+00
Epoch 79/100
378/378 [==============================] - 5s 14ms/step - loss: 36496350.7022 - acc: 0.0000e+00
Epoch 80/100
378/378 [==============================] - 5s 14ms/step - loss: 36780157.9992 - acc: 0.0000e+00
Epoch 81/100
378/378 [==============================] - 5s 14ms/step - loss: 34659411.5025 - acc: 0.0000e+00
Epoch 82/100
378/378 [==============================] - 5s 14ms/step - loss: 34011194.8564 - acc: 0.0000e+00
Epoch 83/100
378/378 [==============================] - 5s 14ms/step - loss: 28422223.4468 - acc: 0.0000e+00
Epoch 84/100
378/378 [==========

In [38]:
y_predict = model.predict(x_test)

In [39]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict): # 실제 정답값, 모델을 통한 예측값 
    return np.sqrt(mean_squared_error(y_test, y_predict))
print('RMSE :', RMSE(y_test, y_predict))

RMSE : 2557.0859815925473


In [40]:
print(model.predict(final_predict_data))

[[55333.98]]
